In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Mar 21 20:38:19 2022

本文件仿真有泵浦光补偿的情况，假设有未知参数，进行逆向研究

@author: Administrator
"""

"""Backend supported: tensorflow.compat.v1, torch"""
import deepxde as dde
import numpy as np
import pandas as pd
#from deepxde.backend import tf
#from deepxde.backend import torch
import torch
from scipy.special import gamma
from scipy.special import lambertw

#import mayavi.mlab as mlab
import matplotlib.pyplot as plt

from draw_counter import random_sphere_points, uniform_sample_points
from PIL import Image

torch.set_default_tensor_type(torch.cuda.FloatTensor)

## 定义模型变量
q_lambda = dde.Variable(1.0)

## 单位统一至cm，定义各个参数
D = 0.585
Rsd = 17.5573
#Rwall = 2.654 # 泡壁碰撞的参数是不是错啦
Rwall = 2.56401
rpump = 0.75
q = 5
#qlambda_real = 0.5
rcell = 1.5

#Rrel = Rsd + (q+q_lambda)*Rwall
I = 25*1e-3*1e4 # kg*(cm)^2/s^3
T = 170
k1 = 10**(-20)*10**8 # kg*cm^4/s^2
k2 = 10**(-17)*10**4 # cm^2
k3 = 1 # cm^-3
sigm = 2.34601*10**(-13) # cm^2

n = k3*(1/(T+273.15))*10**(21.866+4.402-4453/(T+273.15))





Using backend: pytorch



In [2]:

def gen_Rpoints():
    z = 0
    Rz = I/10 # 和球心处的光强成正比
    x_axis = []
    y_axis = []
    Pz_axis = []
    for theta in np.linspace(0, 2*np.pi, 100):
        for rz_power in np.linspace(0, 1.5**2, 100):
            rz = np.sqrt(rz_power) # 均匀采样
            x = rz * np.cos(theta)
            y = rz * np.sin(theta)
            x_axis.append(x)
            y_axis.append(y)
            Pz = Rz*np.exp(-2*(x**2+y**2)/rpump**2)
            Pz_axis.append(Pz)
    z_axis = np.zeros_like(x_axis)        
    position = np.stack((x_axis, y_axis, z_axis), axis=1)   
    Pz_axis = np.asarray(Pz_axis).reshape(-1, 1)
    return position,Pz_axis
# a,b = gen_Rpoints()
# print(a.shape)
# b

In [3]:
# 输入通道3，输出通道3——（Pz, R1, R2）

def bloch_pde(x_in, y_in):
    ''' x_in: (x,y,z) axis
        y_in: Pz polarization, R1, R2
    '''
    #x_in_copy = x_in.detach().cpu().numpy()
    x = x_in[:, 0:1]
    y = x_in[:, 1:2]
    z = x_in[:, 2:3]
    
    Pz = y_in[:, 0:1]
    R1 = y_in[:, 1:2]
    R2 = y_in[:, 2:3]
    
    dy_xx = dde.grad.hessian(y_in, x_in, i=0, j=0, component=0)
    dy_yy = dde.grad.hessian(y_in, x_in, i=1, j=1, component=0)
    dy_zz = dde.grad.hessian(y_in, x_in, i=2, j=2, component=0) # dy/dxidxj
    
    Rop = y_in[:, 1:2] + y_in[:, 2:3] ## 定义Rop = R1 + R2
    eqn1 = -D*(dy_xx+dy_yy+dy_zz) + ((Rop+Rsd)/(q)+Rwall)*Pz - Rop/(q)
    ## 设定R1和R2的约束方程
    dR1_z = dde.grad.jacobian(y_in, x_in, i=1, j=2) # 可以确认下~
    dR2_z = dde.grad.jacobian(y_in, x_in, i=2, j=2)
    
    eqn2 = dR1_z + n*R1*(1-Pz)
    eqn3 = dR2_z - n*R2*(1-Pz)
    
    return [eqn1, eqn2, eqn3]

# def func(x):
#     return (np.abs(1 - np.linalg.norm(x, axis=1, keepdims=True) ** 2)) ** (
#         1 + alpha / 2
#     )

def func_boundary(x_in, y_in, numpy_X):
    #x_in_copy = x_in.detach().cpu().numpy()
    x = x_in[:, 0:1]
    y = x_in[:, 1:2]
    z = x_in[:, 2:3]
    
    Pz = y_in[:, 0:1]
    R1 = y_in[:, 1:2]
    R2 = y_in[:, 2:3]
    
    dPz_x = dde.grad.jacobian(y_in, x_in, i=0, j=0) # This is right
    dPz_y = dde.grad.jacobian(y_in, x_in, i=0, j=1)
    dPz_z = dde.grad.jacobian(y_in, x_in, i=0, j=2)
    
    #Rop = Rop0 * np.exp(-2*(x**2+y**2)/rpump**2)
#     q_copy = q_lambda.detach().cpu().numpy()
#     Rrel = Rsd + (q+q_copy)*Rwall
#     plog1 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))-n*sigm*z))
#     plog2 = lambertw((sigm*I/(Rrel*k1))*np.exp((sigm*I/(Rrel*k1))+n*sigm*z))
#     Rop0_z = k2*(Rrel/sigm)*(np.real(plog1) + np.real(plog2))
#     Rop0_z = np.asarray(Rop0_z)      
    
#     Rop = Rop0_z * np.exp(-2*(x**2+y**2)/rpump**2)
    Rop = R1 + R2
    
    second_term = torch.sqrt(1/2*torch.abs(Rop*D))
    
    # tranfer back to torch.tensor
    #second_term = torch.from_numpy(second_term).cuda()
    #Rop = torch.from_numpy(Rop).cuda()
#     x = torch.from_numpy(x).cuda()
#     y = torch.from_numpy(y).cuda()
#     z = torch.from_numpy(z).cuda()
    
    return (D/rcell*(x*dPz_x + y*dPz_y + z*dPz_z) + Pz*second_term)

In [4]:
geom = dde.geometry.Sphere([0, 0, 0], rcell)
#bc = dde.icbc.DirichletBC(geom, func_boundary, lambda _, on_boundary: on_boundary)

#bc = dde.icbc.RobinBC(geom, func_boundary, lambda _, on_boundary: on_boundary)
bc = dde.icbc.OperatorBC(geom, func_boundary, lambda _, on_boundary: on_boundary) # 函数调研下，operatorBC


## 添加变量参数约束

# Get the train data
observe_axis, ob_pz = gen_Rpoints()
print('Pz shape ', ob_pz.shape)
observe_points_R1 = dde.icbc.PointSetBC(observe_axis, ob_pz, component=1)
observe_points_R2 = dde.icbc.PointSetBC(observe_axis, ob_pz, component=2)


data = dde.data.PDE(geom, bloch_pde,\
                    [bc, observe_points_R1, observe_points_R2],\
                    num_domain=25000, num_boundary=3000,\
                   anchors=observe_axis)

net = dde.nn.FNN([3] + [60] * 4 + [3], "tanh", "Glorot normal")
# net.apply_output_transform(
#     lambda x, y: (1 - tf.reduce_sum(x ** 2, axis=1, keepdims=True)) * y
# )

model = dde.Model(data, net)
#model.compile("adam", lr=1e-4, external_trainable_variables=[q_lambda])
model.compile("adam", lr=1e-4)
#losshistory, train_state = model.train(epochs=8000, model_save_path='Bloch_Rop_v1')
# variable = dde.callbacks.VariableValue(
#     [q_lambda], period=500, filename="variables.dat"
# )
# losshistory, train_state = model.train(iterations=60000, callbacks=[variable])
# dde.saveplot(losshistory, train_state, issave=True, isplot=True)
# losshistory, train_state = model.train(epochs=30000, callbacks=[variable])
# dde.saveplot(losshistory, train_state, issave=True, isplot=True)

losshistory, train_state = model.train(epochs=20000)

Pz shape  (10000, 1)


C:\Users\Administrator\anaconda3\envs\tf2\lib\site-packages\skopt\sampler\sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+25002=25002. 
  total_n_samples))
C:\Users\Administrator\anaconda3\envs\tf2\lib\site-packages\skopt\sampler\sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+3002=3002. 
  total_n_samples))


Compiling model...
'compile' took 0.000358 s

Training model...



TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.